In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score

from imblearn.under_sampling import RandomUnderSampler

import tensorflow as tf

In [2]:
# Selecting cleaned data from your local drive
from google.colab import files
uploaded = files.upload()

Saving post_clean.csv to post_clean.csv


In [3]:
import io
df = pd.read_csv(io.BytesIO(uploaded['post_clean.csv']))
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.60,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.89,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.50,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.40,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.00,never smoked,1


In [4]:
# confirming data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5110 non-null   object 
 1   age                5110 non-null   float64
 2   hypertension       5110 non-null   int64  
 3   heart_disease      5110 non-null   int64  
 4   ever_married       5110 non-null   object 
 5   work_type          5110 non-null   object 
 6   Residence_type     5110 non-null   object 
 7   avg_glucose_level  5110 non-null   float64
 8   bmi                5110 non-null   float64
 9   smoking_status     5110 non-null   object 
 10  stroke             5110 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 439.3+ KB


In [5]:

# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df,dtype=float)
df.head()

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Govt_job,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.60,1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,61.0,0,0,202.21,28.89,1,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,80.0,0,1,105.92,32.50,1,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,49.0,0,0,171.23,34.40,1,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,79.0,1,0,174.12,24.00,1,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [6]:

# Split our preprocessed data into our features and target arrays
y = df['stroke']
print(y[:5])
X = df.drop(columns=['stroke'],axis=1)
X.head()

0    1
1    1
2    1
3    1
4    1
Name: stroke, dtype: int64


,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,ever_married_Yes,work_type_Govt_job,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_Unknown,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.60,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,61.0,0,0,202.21,28.89,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,80.0,0,1,105.92,32.50,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,49.0,0,0,171.23,34.40,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4,79.0,1,0,174.12,24.00,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,stratify=y)


In [8]:
# Creating Random Under Sampler to balance dataset
rus=RandomUnderSampler(sampling_strategy=.15,random_state=2)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)


In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train_resampled)

# Scale the data
X_train_r_scaled = X_scaler.transform(X_train_resampled)
X_test_scaled = X_scaler.transform(X_test)

<b>Testing for Tuner:</b>

In [10]:
input_features = len(X_train_r_scaled[0])
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','leaky_relu'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=20,
        step=5), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=16,
            step=5),
            activation=activation))
    # Output layer defined by sigmoid
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    # nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    
    return nn_model

In [11]:
tf.keras.backend.clear_session()

In [12]:
# Import the kerastuner library
!pip install keras-tuner --upgrade
import keras_tuner as kt
tuner = kt.Hyperband(
    create_model,
    objective=kt.Objective("val_auc", direction="max"),
    max_epochs=25,
    hyperband_iterations=2)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.7 MB/s eta 0:00:00


In [13]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_r_scaled,y_train_resampled,epochs=25,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 12s]
val_auc: 0.8002308011054993

Best val_auc So Far: 0.8230183124542236
Total elapsed time: 00h 05m 35s


In [14]:

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 1,
 'num_layers': 1,
 'units_0': 11,
 'units_1': 6,
 'units_2': 11,
 'units_3': 6,
 'units_4': 16,
 'units_5': 16,
 'tuner/epochs': 25,
 'tuner/initial_epoch': 9,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0048'}

In [15]:
# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 1, 'num_layers': 1, 'units_0': 11, 'units_1': 6, 'units_2': 11, 'units_3': 6, 'units_4': 16, 'units_5': 16, 'tuner/epochs': 25, 'tuner/initial_epoch': 9, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0048'}
{'activation': 'tanh', 'first_units': 1, 'num_layers': 2, 'units_0': 1, 'units_1': 11, 'units_2': 6, 'units_3': 16, 'units_4': 6, 'units_5': 1, 'tuner/epochs': 25, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'activation': 'leaky_relu', 'first_units': 11, 'num_layers': 3, 'units_0': 6, 'units_1': 16, 'units_2': 1, 'units_3': 6, 'units_4': 6, 'units_5': 16, 'tuner/epochs': 25, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}


In [16]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")



40/40 - 0s - loss: 0.1978 - auc: 0.8230 - 316ms/epoch - 8ms/step
Loss: 0.1977921575307846, Accuracy: 0.8230183124542236
40/40 - 1s - loss: 0.1967 - auc: 0.8223 - 514ms/epoch - 13ms/step
Loss: 0.19668564200401306, Accuracy: 0.8223484754562378
40/40 - 1s - loss: 0.1960 - auc: 0.8157 - 598ms/epoch - 15ms/step
Loss: 0.19597694277763367, Accuracy: 0.8157497048377991


In [17]:
# Evaluate ROC score
for m in top_model:
  R_S= roc_auc_score(y_test,m.predict(X_test_scaled))
  print(R_S)


40/40 [==============================] - 1s 6ms/step
0.8218113327674024
40/40 [==============================] - 1s 6ms/step
0.8233632215619695
40/40 [==============================] - 1s 6ms/step
0.8152987054329371


In [18]:
# Setting up confusion matrix
best_m = tuner.get_best_models(1)[0]

index= ["Actual No Stroke", "Actual Stroke"]
columns= ["Predicted No Stroke","Predicted Stroke"]
predictions_4 = best_m.predict(X_test_scaled)
threshold = (predictions_4>0.5).astype(int)
cm_3= confusion_matrix(y_test, threshold)
pd.DataFrame(cm_3,index=index,columns=columns)

40/40 [==============================] - 0s 3ms/step


,Predicted No Stroke,Predicted Stroke
Actual No Stroke,1210,6
Actual Stroke,59,3


In [19]:
# Classification Matrix
print(classification_report(y_test,threshold))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1216
           1       0.33      0.05      0.08        62

    accuracy                           0.95      1278
   macro avg       0.64      0.52      0.53      1278
weighted avg       0.92      0.95      0.93      1278



In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_dimen= len(X_train_r_scaled[0])
nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units= 16,activation='relu',input_dim= input_dimen))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units= 16,activation='relu'))

# Third hidden layer
nn4.add(tf.keras.layers.Dense(units= 16,activation='relu'))

# Fourth hidden layer
nn4.add(tf.keras.layers.Dense(units= 16,activation='relu'))

# Fifth hidden layer
nn4.add(tf.keras.layers.Dense(units= 16,activation='relu'))

# Fifth hidden layer
nn4.add(tf.keras.layers.Dense(units= 16,activation='relu'))

# Output layer
nn4.add(tf.keras.layers.Dense(units= 1,activation='sigmoid'))

# Check the structure of the model
nn4.summary()

# Compile the model
nn4.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 16)                336       
                                                                 
 dense_11 (Dense)            (None, 16)                272       
                                                                 
 dense_12 (Dense)            (None, 16)                272       
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 16)                272       
                                                                 
 dense_15 (Dense)            (None, 16)                272       
                                                                 
 dense_16 (Dense)            (None, 1)                

In [25]:
# Training best Model
fit_model= nn4.fit(X_train_r_scaled,y_train_resampled,epochs=120)

Epoch 1/120
45/45 [==============================] - 2s 3ms/step - loss: 0.4278 - accuracy: 0.8695
Epoch 2/120
45/45 [==============================] - 0s 3ms/step - loss: 0.3570 - accuracy: 0.8695
Epoch 3/120
45/45 [==============================] - 0s 3ms/step - loss: 0.3371 - accuracy: 0.8695
Epoch 4/120
45/45 [==============================] - 0s 3ms/step - loss: 0.3246 - accuracy: 0.8695
Epoch 5/120
45/45 [==============================] - 0s 3ms/step - loss: 0.3117 - accuracy: 0.8695
Epoch 6/120
45/45 [==============================] - 0s 3ms/step - loss: 0.3024 - accuracy: 0.8695
Epoch 7/120
45/45 [==============================] - 0s 3ms/step - loss: 0.2952 - accuracy: 0.8695
Epoch 8/120
45/45 [==============================] - 0s 3ms/step - loss: 0.2875 - accuracy: 0.8716
Epoch 9/120
45/45 [==============================] - 0s 3ms/step - loss: 0.2817 - accuracy: 0.8765
Epoch 10/120
45/45 [==============================] - 0s 3ms/step - loss: 0.2753 - accuracy: 0.8758
Epoch 11/

In [26]:
# Evaluate the model using the test data
model_loss_4, model_accuracy_4 = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss_4}, Accuracy: {model_accuracy_4}")

40/40 - 0s - loss: 0.6684 - accuracy: 0.8654 - 352ms/epoch - 9ms/step
Loss: 0.668429434299469, Accuracy: 0.8654147386550903


In [27]:
# Checking ROC Score
roc_4 = roc_auc_score(y_test,nn4.predict(X_test_scaled))
print(roc_4)

40/40 [==============================] - 0s 3ms/step
0.6387547750424447


In [28]:
# Confusion Matrix
index= ["Actual No Stroke", "Actual Stroke"]
columns= ["Predicted No Stroke","Predicted Stroke"]
predictions_4 = nn4.predict(X_test_scaled)
threshold_2 = (predictions_4>0.5).astype(int)
cm_3= confusion_matrix(y_test, threshold_2)
pd.DataFrame(cm_3,index=index,columns=columns)

40/40 [==============================] - 0s 2ms/step


,Predicted No Stroke,Predicted Stroke
Actual No Stroke,1091,125
Actual Stroke,47,15


In [29]:
# Classification Matrix
print(classification_report(y_test,threshold_2))

              precision    recall  f1-score   support

           0       0.96      0.90      0.93      1216
           1       0.11      0.24      0.15        62

    accuracy                           0.87      1278
   macro avg       0.53      0.57      0.54      1278
weighted avg       0.92      0.87      0.89      1278

